In [2]:
 #!pip install pysam

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.0/22.0 MB 36.8 MB/s eta 0:00:00


In [6]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pysam
import gzip
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [8]:
def read_vcf(file_path):
    """
    Reads a VCF file and returns a DataFrame containing the data.
    """
    vcf_in = pysam.VariantFile(file_path)  # auto-detect input format
    records = []

    for record in vcf_in:
        record_dict = {
            "CHROM": record.chrom,
            "POS": record.pos,
            "ID": record.id,
            "REF": record.ref,
            "ALT": ','.join([str(alt) for alt in record.alts]),
            "QUAL": record.qual,
            "FILTER": ','.join(record.filter.keys()) if record.filter else 'PASS',
            "INFO": record.info
        }
        records.append(record_dict)

    df = pd.DataFrame(records)
    return df

def process_info_column(df):
    """
    Processes the INFO column to expand it into separate columns.
    """
    info_df = df['INFO'].apply(pd.Series)
    df = pd.concat([df.drop(['INFO'], axis=1), info_df], axis=1)
    return df

def plot_quality_distribution(df):
    """
    Plots the distribution of quality scores.
    """
    plt.hist(df['QUAL'].dropna(), bins=50, color='c', edgecolor='k', alpha=0.65)
    plt.title('Quality Score Distribution')
    plt.xlabel('Quality Score')
    plt.ylabel('Frequency')
    plt.show()

def plot_variant_types(df):
    """
    Plots the distribution of variant types (SNPs vs. INDELs).
    """
    df['VARIANT_TYPE'] = df.apply(lambda row: 'SNP' if len(row['REF']) == 1 and len(row['ALT']) == 1 else 'INDEL', axis=1)
    variant_counts = df['VARIANT_TYPE'].value_counts()
    variant_counts.plot(kind='bar', color='skyblue', edgecolor='black')
    plt.title('Variant Type Distribution')
    plt.xlabel('Variant Type')
    plt.ylabel('Frequency')
    plt.show()

In [7]:
directory = '/content/drive/MyDrive/Personal Projects/MaizeEvolutionaryFitness/Trial Raw Data'
file_paths = glob.glob(os.path.join(directory, '*.vcf.gz'))

In [ ]:
count = 0
for file_path in file_paths:
  count += 1
  print(f"Processing file: {file_path}")

  # Extract the file
  with gzip.open(file_path, 'rb') as f_in:
      with open(file_path[:-3], 'wb') as f_out:
          f_out.write(f_in.read())

  # Read VCF file
  df = read_vcf(file_path[:-3])
  os.remove(file_path[:-3])  # Remove extracted file

  # Process DataFrame
  df = process_info_column(df)

  print(df.head())

  plot_quality_distribution(df)
  plot_variant_types(df)

  if count == 1:
    break

Processing file: /content/drive/MyDrive/Personal Projects/MaizeEvolutionaryFitness/Trial Raw Data/chr_1_imputed.vcf.gz
